<a href="https://colab.research.google.com/github/nnarey/NCAABB-Score-Predictor/blob/main/Score_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

## **Setup**

# pd.set_option('display.max_columns', None)

(0) Setup (imports, options)

(1) Define Helper Functions

(2) Define Main Functions (aggregate_by_date), train

(3) Main (Running functions)

In [2]:
pd.set_option('display.max_columns', None)


In [3]:
import pandas as pd
df = pd.read_csv('/content/bq-results-20240731-211713-1722460653258 (1).csv')
df["ptsDiff"] = df["ptsScored"] - df["ptsAgst"]
df = df[df["isOffense"] == True]
df.head(2)

,_id,isOffense,teamId,teamIdAgst,gameId,competitionId,leagueId,inConferenceReg,inConferenceAll,isPostSeason,gameTime,gameDate,gameStatus,isNeutral,isLiveStream,tournamentId,homeId,awayId,period,updated,scp,drb,orb,pitp,potov,fta,ftm,fbpts,tmDrb,tmOrb,tov,fgm3,tmTov,fga2,fgm2,fga3,stl,tf,ast,benchPts,blk,blkd,pfd,opf,pf,dpf,fga,fgm,reb,tmReb,fg3Pct,fg2Pct,fgPct,ptsScored,ftPct,astTov,ptsAgst,mins,biggestRunScore,biggestRun,biggestLeadScore,timeLeading,leadChanges,timesTied,biggestLead,ptsP1,ptsP2,ptsP3,ptsP4,ptsOt,drbAgst,orbAgst,rebAgst,fgmAgst,fgaAgst,fgm2Agst,fga2Agst,fgm3Agst,fga3Agst,ftmAgst,ftaAgst,tovAgst,biggestLeadAgst,biggestRunAgst,oPoss,dPoss,tsa,poss,pace,fga3Rate,efgPct,tsPct,ftaRate,ftmRate,astPct,astRatio,blkPct,stlPct,rebPct,drbPct,orbPct,tovPct,tovDiff,rebDiff,rebTovDiff,fgmDiff,fgaDiff,fgm2Diff,fga2Diff,fgm3Diff,fga3Diff,ftmDiff,ftaDiff,pfEff,stlPerPf,blkPerPf,pfdP100,scpPctPts,fbptsPctPts,pitpPctPts,ortg,drtg,eff,hkmPct,netRtg,teamMarket,teamName,conferenceId,divisionId,teamMarketAgst,conferenceIdAgst,divisionIdAgst,teamGameRecency,isExhib,inDivision,isHome,netRankAgst,quadAgst,overallWins,overallLosses,confWins,confLosses,gameUpdated,ptsDiff
0,2346638-REGULAR-0-104196-true,True,104196,104450,2346638,36046,456,True,True,False,2024-02-14 20:00:00 UTC,2024-02-14,COMPLETE,False,True,0,104196,104450,game,2024-02-15 04:12:27 UTC,11,25,9,36,15,28,23,9,1,4,9,9,1,34,19,22,7,0,15,30,7,4,23,3,19,16,56,28,34,5,0.409091,0.558824,0.500000,88,0.821429,1.666667,62,40.0,47-32,9.0,88 - 62,34.266667,5.0,1.0,26.0,38,50,NaN,NaN,NaN,21,13,34,21,57,17,39,4,18,16,19,16,4.0,7.0,67.21200,67.42211,68.3,67.31706,67.317,0.39286,0.58036,0.64422,0.50000,0.41071,0.53571,0.16248,0.17949,0.10382,0.5,0.65789,0.30000,0.11640,-7,0,7,7,-1,2,-5,5,4,7,9,0.73684,0.36842,0.36842,34.16667,0.12500,0.10227,0.40909,130.72466,92.10147,109,0.2833,38.6232,Samford,Bulldogs,38,1,Western Caro.,38,1,10,False,True,True,111,quad3,23,3,12,1,2024-02-15 04:12:36 UTC,26
2,2346638-REGULAR-0-104450-true,True,104450,104196,2346638,36046,456,True,True,False,2024-02-14 20:00:00 UTC,2024-02-14,COMPLETE,False,True,0,104196,104450,game,2024-02-15 04:12:28 UTC,10,21,13,32,7,19,16,5,4,2,16,4,0,39,17,18,3,0,8,13,4,7,19,2,23,21,57,21,34,6,0.222222,0.435897,0.368421,62,0.842105,0.500000,88,40.0,7-11,7.0,0 - 4,4.650000,5.0,1.0,4.0,29,33,NaN,NaN,NaN,25,9,34,28,56,19,34,9,22,23,28,9,26.0,9.0,67.42211,67.21200,65.4,67.31706,67.317,0.31579,0.40351,0.47401,0.33333,0.28070,0.38095,0.08953,0.11765,0.04463,0.5,0.70000,0.34211,0.19666,7,0,-7,-7,1,-2,5,-5,-4,-7,-9,0.30435,0.13043,0.17391,28.22464,0.16129,0.08065,0.51613,92.10147,130.72466,45,0.1623,-38.6232,Western Caro.,Catamounts,38,1,Samford,38,1,7,False,True,False,74,quad1,18,8,7,6,2024-02-15 04:12:36 UTC,-26


In [4]:
def aggregate_stats_through_date(gameData, endDate):
    """
    Filters the game data based on the endDate and aggregates the game stats into season stats.

    Parameters:
    gameData (pd.DataFrame): DataFrame containing game data with relevant statistics.
    endDate (str): The end date (in 'YYYY-MM-DD' format) up to which the games should be considered.

    Returns:
    pd.DataFrame: A DataFrame containing the aggregated season statistics.
    """

    # Step 1: Convert the 'Date' column to datetime format
    gameData['gameTime'] = pd.to_datetime(gameData['gameTime'])

    # Step 2: Convert endDate to datetime format
    endDate = pd.to_datetime(endDate)

    # Step 3: Filter the game data to include only games up to the given endDate
    filteredData = gameData[gameData['gameTime'] <= endDate]

    # Step 4: Aggregate the stats
    seasonStats = filteredData.groupby('teamMarket').agg({
        'ptsScored': 'sum',
        'ptsAgst': 'sum',
        'tovDiff': 'sum',
        'ast': 'sum',
        'reb': 'sum',
        'stl': 'sum',
        'blk': 'sum',
        'tov': 'sum',
        'orb': 'sum',
        'drb': 'sum',
        'pf': 'sum',
        'fgm': 'sum',
        'fga': 'sum',
        'fgm3': 'sum',
        'fga3': 'sum',
        'ftm': 'sum',
        'fta': 'sum',
        'ptsP1': 'sum',
        'ptsP2': 'sum',
        'ptsDiff': 'sum',
        'ortg': 'mean',
        'drtg': 'mean',
        'netRankAgst': 'mean'
    }).reset_index()

    # Step 5: Calculate derived stats
    seasonStats = seasonStats.assign(
        fgPct = seasonStats['fgm'] / seasonStats['fga'],
        fg3Pct = seasonStats['fgm3'] / seasonStats['fga3'],
        ftPct = seasonStats['ftm'] / seasonStats['fta'],
        tsPct = seasonStats['ptsScored'] / (2 * (seasonStats['fga'] + 0.44 * seasonStats['fta'])),
        efgPct = (seasonStats['fgm'] + 0.5 * seasonStats['fgm3']) / seasonStats['fga'],
        tovPct = 100 * seasonStats['tov'] / (seasonStats['fga'] + 0.44 * seasonStats['fta'] + seasonStats['tov']),
        orbPct = seasonStats['orb'] / (seasonStats['orb'] + seasonStats['drb']),  # Assuming drbAgainst is not available
        drbPct = seasonStats['drb'] / (seasonStats['drb'] + seasonStats['orb']),  # Assuming orbAgainst is not available
        astTov = seasonStats['ast'] / seasonStats['tov'],
        tovDiff = seasonStats['tov'] - seasonStats['tovDiff']  # Assuming tovDiff is calculated this way
    )

    # Step 6: Return the aggregated season stats
    return seasonStats


In [5]:
aggregate_stats_through_date(df, "2023-03-03 00:00:00 UTC")

,teamMarket,ptsScored,ptsAgst,tovDiff,ast,reb,stl,blk,tov,orb,drb,pf,fgm,fga,fgm3,fga3,ftm,fta,ptsP1,ptsP2,ptsDiff,ortg,drtg,netRankAgst,fgPct,fg3Pct,ftPct,tsPct,efgPct,tovPct,orbPct,drbPct,astTov
0,A&M-Corpus Christi,4395,4189,915,828,2129,487,92,795,707,1422,1187,1506,3484,398,1163,985,1281,2026,2331,206,107.541803,102.349132,268.482759,0.432262,0.342218,0.768931,0.542909,0.489380,16.416665,0.332081,0.667919,1.041509
1,Abilene Christian,4099,4031,1043,811,1714,530,105,704,541,1173,1216,1415,3279,388,1152,881,1189,1899,2144,68,104.599642,103.003874,175.053571,0.431534,0.336806,0.740959,0.539036,0.490698,15.623058,0.315636,0.684364,1.151989
2,Air Force,3723,3920,700,826,1641,334,213,760,363,1278,1032,1338,2958,455,1313,592,862,1728,1967,-197,100.707961,106.342445,158.898305,0.452333,0.346535,0.686775,0.557790,0.529243,18.548891,0.221207,0.778793,1.086842
3,Akron,4242,3919,696,724,2077,339,169,670,587,1490,945,1453,3271,488,1396,848,1184,1988,2236,323,110.226384,102.020349,205.016667,0.444207,0.349570,0.716216,0.559341,0.518802,15.015823,0.282619,0.717381,1.080597
4,Alabama,5130,4612,794,937,2636,417,306,905,818,1818,1173,1751,3933,607,1863,1021,1405,2387,2687,518,112.195560,100.609202,75.253968,0.445207,0.325819,0.726690,0.563588,0.522375,16.586635,0.310319,0.689681,1.035359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Wright St.,5070,4849,847,945,2329,392,191,825,642,1687,1003,1902,4013,418,1223,848,1120,2382,2634,221,109.821708,104.592453,228.939394,0.473960,0.341783,0.757143,0.562608,0.526040,15.476101,0.275655,0.724345,1.145455
359,Wyoming,4274,4179,656,657,2065,253,137,718,489,1576,942,1474,3285,483,1431,843,1171,1990,2229,95,107.112779,104.773913,125.803279,0.448706,0.337526,0.719898,0.562333,0.522222,15.891143,0.236804,0.763196,0.915042
360,Xavier,5136,4735,822,1132,2459,470,258,806,677,1782,1039,1868,3945,474,1332,926,1309,2428,2663,401,111.129482,102.784731,92.954545,0.473511,0.355856,0.707410,0.568021,0.533587,15.130581,0.275315,0.724685,1.404467
361,Yale,3912,3622,690,691,1965,328,182,653,504,1461,955,1420,3143,374,1111,698,972,1793,2073,290,106.607701,98.377464,185.055556,0.451798,0.336634,0.718107,0.547795,0.511295,15.460452,0.256489,0.743511,1.058193


In [6]:
def aggregate_over_all_dates(gameData):
    """
    Aggregates stats for each team through all unique game dates in the season.

    Parameters:
    gameData (pd.DataFrame): DataFrame containing game data with relevant statistics.

    Returns:
    pd.DataFrame: A DataFrame containing aggregated season statistics for each team on each game date.
    """

    # Step 1: Get a unique list of all gameDates in the season
    gameData['gameTime'] = pd.to_datetime(gameData['gameTime'])  # Ensure gameTime is in datetime format
    unique_dates = gameData['gameTime'].sort_values().unique()  # Unique list of game dates, sorted

    # Step 2: Initialize an empty DataFrame to store results
    season_df_stacked_by_date = pd.DataFrame()

    # Step 3: Loop through each date and call aggregate_stats_through_date() for the date
    for date in unique_dates:
        aggregated_stats = aggregate_stats_through_date(gameData, date)

        # Add the endDate (the current date) to the aggregated data
        aggregated_stats['endDate'] = date

        # Append the aggregated data for this date to the final DataFrame
        season_df_stacked_by_date = pd.concat([season_df_stacked_by_date, aggregated_stats], ignore_index=True)

    # Step 4: Return the DataFrame that contains the stats for each team through each date
    return season_df_stacked_by_date


In [7]:
all_dates = aggregate_over_all_dates(df)
all_dates

,teamMarket,ptsScored,ptsAgst,tovDiff,ast,reb,stl,blk,tov,orb,drb,pf,fgm,fga,fgm3,fga3,ftm,fta,ptsP1,ptsP2,ptsDiff,ortg,drtg,netRankAgst,fgPct,fg3Pct,ftPct,tsPct,efgPct,tovPct,orbPct,drbPct,astTov,endDate
0,Alcorn,67,85,14,7,27,5,3,12,11,16,28,23,60,4,17,17,21,25,42,-18,100.170840,127.082410,54.000000,0.383333,0.235294,0.809524,0.483824,0.416667,14.771049,0.407407,0.592593,0.583333,2021-11-09 12:00:00+00:00
1,Washington St.,85,67,12,16,46,5,4,14,19,27,20,25,58,11,36,24,30,39,46,18,127.082410,100.170840,262.000000,0.431034,0.305556,0.800000,0.596910,0.525862,16.431925,0.413043,0.586957,1.142857,2021-11-09 12:00:00+00:00
2,Alcorn,67,85,14,7,27,5,3,12,11,16,28,23,60,4,17,17,21,25,42,-18,100.170840,127.082410,54.000000,0.383333,0.235294,0.809524,0.483824,0.416667,14.771049,0.407407,0.592593,0.583333,2021-11-09 14:00:00+00:00
3,California,67,80,13,4,31,5,1,11,10,21,20,24,60,5,18,14,24,37,30,-13,95.814840,114.405770,272.000000,0.400000,0.277778,0.583333,0.474773,0.441667,13.487003,0.322581,0.677419,0.363636,2021-11-09 14:00:00+00:00
4,UC San Diego,80,67,11,10,30,5,2,13,1,29,20,24,46,10,22,22,28,33,47,13,114.405770,95.814840,150.000000,0.521739,0.454545,0.785714,0.685871,0.630435,18.227706,0.033333,0.966667,0.769231,2021-11-09 14:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394631,Wright St.,7655,7332,1209,1435,3357,578,272,1189,918,2439,1482,2861,5828,622,1754,1311,1729,3640,3930,323,112.862246,107.810369,218.812500,0.490906,0.354618,0.758242,0.580914,0.544269,15.287178,0.273458,0.726542,1.206897,2024-04-08 18:20:00+00:00
1394632,Wyoming,6547,6601,998,1016,3153,414,213,1118,795,2358,1455,2247,5049,721,2091,1332,1827,3023,3447,-54,106.976384,107.831406,125.827957,0.445039,0.344811,0.729064,0.559297,0.516439,16.038147,0.252141,0.747859,0.908766,2024-04-08 18:20:00+00:00
1394633,Xavier,8242,7771,1281,1789,4028,741,391,1275,1153,2875,1718,2985,6484,748,2132,1524,2125,3860,4337,471,110.195821,104.095427,91.327103,0.460364,0.350844,0.717176,0.555466,0.518044,14.665286,0.286246,0.713754,1.403137,2024-04-08 18:20:00+00:00
1394634,Yale,6471,6039,1070,1189,3206,537,299,998,832,2374,1509,2361,5196,633,1839,1116,1565,3013,3379,432,108.392845,100.954073,171.393258,0.454388,0.344209,0.713099,0.549825,0.515300,14.500334,0.259513,0.740487,1.191383,2024-04-08 18:20:00+00:00


In [8]:
def create_training_df(gameData, season_df_stacked_by_date):
    """
    Joins the aggregated stats to the game data, providing game info, scores, and season stats up to the game date.

    Parameters:
    gameData (pd.DataFrame): DataFrame containing game information and scores.
    season_df_stacked_by_date (pd.DataFrame): DataFrame containing aggregated team stats up to various dates.

    Returns:
    pd.DataFrame: A DataFrame with game info, scores, and season stats through the game date.
    """

    # Step 1: Create a "skinny" version of the gameData with only the necessary fields
    skinny_gameData = gameData[['competitionId', 'conferenceId', 'teamId', 'gameId', 'gameTime',
                                'teamMarket', 'ptsScored', 'ptsAgst', 'ptsDiff']].copy()

    # Step 2: Ensure 'gameTime' and 'endDate' are in datetime format for proper merging
    skinny_gameData.loc[:, 'gameTime'] = pd.to_datetime(skinny_gameData['gameTime'])
    season_df_stacked_by_date.loc[:, 'endDate'] = pd.to_datetime(season_df_stacked_by_date['endDate'])

    # Step 3: Convert both 'teamId' in skinny_gameData and 'teamMarket' in season_df_stacked_by_date to strings
    skinny_gameData.loc[:, 'teamId'] = skinny_gameData['teamId'].astype(str)
    season_df_stacked_by_date.loc[:, 'teamMarket'] = season_df_stacked_by_date['teamMarket'].astype(str)

    # Step 4: Merge the skinny gameData with the aggregated stats
    training_df = pd.merge(
        skinny_gameData,
        season_df_stacked_by_date,
        left_on=['gameTime', 'teamMarket'],  # gameData's gameTime == season_df's endDate, and teamId matches
        right_on=['endDate', 'teamMarket'],  # season_df's teamMarket is equivalent to teamMarket in gameData
        how='inner'  # Use 'inner' join to ensure only rows with matching game dates and teams are included
    )

    return training_df


In [9]:
training_df = create_training_df(df, all_dates)

In [10]:
training_df

,competitionId,conferenceId,teamId,gameId,gameTime,teamMarket,ptsScored_x,ptsAgst_x,ptsDiff_x,ptsScored_y,ptsAgst_y,tovDiff,ast,reb,stl,blk,tov,orb,drb,pf,fgm,fga,fgm3,fga3,ftm,fta,ptsP1,ptsP2,ptsDiff_y,ortg,drtg,netRankAgst,fgPct,fg3Pct,ftPct,tsPct,efgPct,tovPct,orbPct,drbPct,astTov,endDate
0,36046,38,104196,2346638,2024-02-14 20:00:00+00:00,Samford,88,62,26,6343,5955,1153,1106,2845,668,254,1063,817,2028,1484,2201,4753,745,2042,1196,1629,3063,3228,388,112.020788,105.446404,199.000000,0.463076,0.364838,0.734193,0.579824,0.541448,16.271836,0.287170,0.712830,1.040452,2024-02-14 20:00:00+00:00
1,36046,38,104450,2346638,2024-02-14 20:00:00+00:00,Western Caro.,62,88,-26,6060,6298,854,1018,3109,408,219,951,874,2235,1471,2135,5087,732,2214,1058,1520,2812,3145,-238,104.980200,109.129586,193.729412,0.419697,0.330623,0.696053,0.526426,0.491645,14.179639,0.281119,0.718881,1.070452,2024-02-14 20:00:00+00:00
2,30629,38,104450,1979402,2021-11-21 15:30:00+00:00,Western Caro.,64,53,11,452,476,64,95,245,28,12,98,71,174,119,156,363,76,203,64,103,223,218,-24,103.473410,108.847505,201.333333,0.429752,0.374384,0.621359,0.553487,0.534435,19.355348,0.289796,0.710204,0.969388,2021-11-21 15:30:00+00:00
3,33533,38,104450,2183580,2022-11-19 11:00:00+00:00,Western Caro.,98,91,7,2451,2724,370,453,1274,175,79,442,390,884,600,848,2114,346,1076,409,602,1120,1283,-273,101.829974,113.454004,181.885714,0.401135,0.321561,0.679402,0.515158,0.482971,15.668869,0.306122,0.693878,1.024887,2022-11-19 11:00:00+00:00
4,36046,38,104450,2338023,2023-12-05 19:00:00+00:00,Western Caro.,71,97,-26,4855,5186,703,835,2475,335,162,790,695,1780,1215,1697,4088,592,1847,869,1250,2245,2529,-331,103.683190,110.746413,193.739130,0.415117,0.320520,0.695200,0.523394,0.487524,14.554164,0.280808,0.719192,1.056962,2023-12-05 19:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33827,30629,72,103882,1977495,2021-11-11 19:00:00+00:00,McNeese,61,77,-16,123,163,33,27,75,20,13,34,23,52,41,47,137,13,51,16,29,57,66,-40,78.207970,103.560380,43.000000,0.343066,0.254902,0.551724,0.410657,0.390511,18.502394,0.306667,0.693333,0.794118,2021-11-11 19:00:00+00:00
33828,36046,72,103705,2339546,2023-12-19 19:00:00+00:00,Houston Christian,53,89,-36,4471,5156,782,783,2085,397,150,960,642,1443,1120,1545,3581,422,1298,959,1356,2049,2351,-685,100.221119,115.707982,235.258065,0.431444,0.325116,0.707227,0.535111,0.490366,18.685622,0.307914,0.692086,0.815625,2023-12-19 19:00:00+00:00
33829,36046,72,103992,2329250,2023-12-19 20:30:00+00:00,Nicholls,67,73,-6,4792,4897,979,811,2269,544,229,833,700,1569,1294,1809,4061,506,1525,668,1034,2286,2446,-105,103.773296,105.922831,224.692308,0.445457,0.331803,0.646035,0.530563,0.507757,15.573121,0.308506,0.691494,0.973589,2023-12-19 20:30:00+00:00
33830,30629,72,103724,1977522,2021-11-12 19:30:00+00:00,UIW,60,87,-27,117,162,20,21,45,6,4,42,11,34,29,39,93,16,48,23,25,52,65,-45,89.056830,122.984620,66.000000,0.419355,0.333333,0.920000,0.562500,0.505376,28.767123,0.244444,0.755556,0.500000,2021-11-12 19:30:00+00:00


In [11]:
def combine_rows(gameData):
    """
    Combines rows in the gameData based on the 'competitionId' without aggregating numerical values.
    This results in columns like 'teamMarket_x' and 'teamMarket_y' for teams in the same competition.

    Parameters:
    gameData (pd.DataFrame): DataFrame containing game information.

    Returns:
    pd.DataFrame: A DataFrame with combined rows for each competition, showing team data side by side.
    """

    # Step 1: Merge the DataFrame with itself on 'competitionId'
    combined_df = pd.merge(
        gameData,  # Left side of the merge (original DataFrame)
        gameData,  # Right side of the merge (also original DataFrame)
        on='gameId',  # Joining on 'competitionId'
        suffixes=('_x', '_y')  # This adds _x and _y to distinguish between columns from both DataFrames
    )

    # Step 2: Optionally, filter down the combined DataFrame if needed
    # For example, you may only want rows where the two teams are different:
    # combined_df = combined_df[combined_df['teamId_x'] != combined_df['teamId_y']]
    combined_df = combined_df[combined_df['teamId_x'] != combined_df['teamId_y']]

    return combined_df

In [12]:
small_df = training_df
combined_df = combine_rows(small_df)
combined_df

,competitionId_x,conferenceId_x,teamId_x,gameId,gameTime_x,teamMarket_x,ptsScored_x_x,ptsAgst_x_x,ptsDiff_x_x,ptsScored_y_x,ptsAgst_y_x,tovDiff_x,ast_x,reb_x,stl_x,blk_x,tov_x,orb_x,drb_x,pf_x,fgm_x,fga_x,fgm3_x,fga3_x,ftm_x,fta_x,ptsP1_x,ptsP2_x,ptsDiff_y_x,ortg_x,drtg_x,netRankAgst_x,fgPct_x,fg3Pct_x,ftPct_x,tsPct_x,efgPct_x,tovPct_x,orbPct_x,drbPct_x,astTov_x,endDate_x,competitionId_y,conferenceId_y,teamId_y,gameTime_y,teamMarket_y,ptsScored_x_y,ptsAgst_x_y,ptsDiff_x_y,ptsScored_y_y,ptsAgst_y_y,tovDiff_y,ast_y,reb_y,stl_y,blk_y,tov_y,orb_y,drb_y,pf_y,fgm_y,fga_y,fgm3_y,fga3_y,ftm_y,fta_y,ptsP1_y,ptsP2_y,ptsDiff_y_y,ortg_y,drtg_y,netRankAgst_y,fgPct_y,fg3Pct_y,ftPct_y,tsPct_y,efgPct_y,tovPct_y,orbPct_y,drbPct_y,astTov_y,endDate_y
1,36046,38,104196,2346638,2024-02-14 20:00:00+00:00,Samford,88,62,26,6343,5955,1153,1106,2845,668,254,1063,817,2028,1484,2201,4753,745,2042,1196,1629,3063,3228,388,112.020788,105.446404,199.000000,0.463076,0.364838,0.734193,0.579824,0.541448,16.271836,0.287170,0.712830,1.040452,2024-02-14 20:00:00+00:00,36046,38,104450,2024-02-14 20:00:00+00:00,Western Caro.,62,88,-26,6060,6298,854,1018,3109,408,219,951,874,2235,1471,2135,5087,732,2214,1058,1520,2812,3145,-238,104.980200,109.129586,193.729412,0.419697,0.330623,0.696053,0.526426,0.491645,14.179639,0.281119,0.718881,1.070452,2024-02-14 20:00:00+00:00
2,36046,38,104450,2346638,2024-02-14 20:00:00+00:00,Western Caro.,62,88,-26,6060,6298,854,1018,3109,408,219,951,874,2235,1471,2135,5087,732,2214,1058,1520,2812,3145,-238,104.980200,109.129586,193.729412,0.419697,0.330623,0.696053,0.526426,0.491645,14.179639,0.281119,0.718881,1.070452,2024-02-14 20:00:00+00:00,36046,38,104196,2024-02-14 20:00:00+00:00,Samford,88,62,26,6343,5955,1153,1106,2845,668,254,1063,817,2028,1484,2201,4753,745,2042,1196,1629,3063,3228,388,112.020788,105.446404,199.000000,0.463076,0.364838,0.734193,0.579824,0.541448,16.271836,0.287170,0.712830,1.040452,2024-02-14 20:00:00+00:00
5,30629,38,104450,1979402,2021-11-21 15:30:00+00:00,Western Caro.,64,53,11,452,476,64,95,245,28,12,98,71,174,119,156,363,76,203,64,103,223,218,-24,103.473410,108.847505,201.333333,0.429752,0.374384,0.621359,0.553487,0.534435,19.355348,0.289796,0.710204,0.969388,2021-11-21 15:30:00+00:00,30629,37,103826,2021-11-21 15:30:00+00:00,Longwood,53,64,-11,290,267,49,55,164,25,8,41,58,106,60,117,268,32,98,24,40,141,149,23,109.994413,98.730955,217.250000,0.436567,0.326531,0.600000,0.507703,0.496269,12.553582,0.353659,0.646341,1.341463,2021-11-21 15:30:00+00:00
6,30629,37,103826,1979402,2021-11-21 15:30:00+00:00,Longwood,53,64,-11,290,267,49,55,164,25,8,41,58,106,60,117,268,32,98,24,40,141,149,23,109.994413,98.730955,217.250000,0.436567,0.326531,0.600000,0.507703,0.496269,12.553582,0.353659,0.646341,1.341463,2021-11-21 15:30:00+00:00,30629,38,104450,2021-11-21 15:30:00+00:00,Western Caro.,64,53,11,452,476,64,95,245,28,12,98,71,174,119,156,363,76,203,64,103,223,218,-24,103.473410,108.847505,201.333333,0.429752,0.374384,0.621359,0.553487,0.534435,19.355348,0.289796,0.710204,0.969388,2021-11-21 15:30:00+00:00
9,33533,38,104450,2183580,2022-11-19 11:00:00+00:00,Western Caro.,98,91,7,2451,2724,370,453,1274,175,79,442,390,884,600,848,2114,346,1076,409,602,1120,1283,-273,101.829974,113.454004,181.885714,0.401135,0.321561,0.679402,0.515158,0.482971,15.668869,0.306122,0.693878,1.024887,2022-11-19 11:00:00+00:00,33533,72,103791,2022-11-19 11:00:00+00:00,Lamar University,91,98,-7,1874,2248,369,347,1020,186,95,434,330,690,547,705,1729,152,520,312,450,854,1014,-374,93.386340,112.004013,202.266667,0.407750,0.292308,0.693333,0.486248,0.451706,18.382042,0.323529,0.676471,0.799539,2022-11-19 11:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

features = ["tsPct_x", "tsPct_y", "tovPct_x", "tovPct_y", "astTov_x", "astTov_y", "ortg_x", "ortg_y", "drtg_x", "drtg_y", "netRankAgst_x", "netRankAgst_y"]

pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=30))  # You can adjust the number of neighbors
X_train = combined_df[features]
y_train = combined_df["ptsDiff_x_x"]

In [14]:
pipeline.fit(X=X_train, y=y_train)
pred = pipeline.predict(X=X_train)
# Add predictions to the DataFrame
combined_df.loc[X_train.index, 'Predicted Differential'] = pred
combined_df

,competitionId_x,conferenceId_x,teamId_x,gameId,gameTime_x,teamMarket_x,ptsScored_x_x,ptsAgst_x_x,ptsDiff_x_x,ptsScored_y_x,ptsAgst_y_x,tovDiff_x,ast_x,reb_x,stl_x,blk_x,tov_x,orb_x,drb_x,pf_x,fgm_x,fga_x,fgm3_x,fga3_x,ftm_x,fta_x,ptsP1_x,ptsP2_x,ptsDiff_y_x,ortg_x,drtg_x,netRankAgst_x,fgPct_x,fg3Pct_x,ftPct_x,tsPct_x,efgPct_x,tovPct_x,orbPct_x,drbPct_x,astTov_x,endDate_x,competitionId_y,conferenceId_y,teamId_y,gameTime_y,teamMarket_y,ptsScored_x_y,ptsAgst_x_y,ptsDiff_x_y,ptsScored_y_y,ptsAgst_y_y,tovDiff_y,ast_y,reb_y,stl_y,blk_y,tov_y,orb_y,drb_y,pf_y,fgm_y,fga_y,fgm3_y,fga3_y,ftm_y,fta_y,ptsP1_y,ptsP2_y,ptsDiff_y_y,ortg_y,drtg_y,netRankAgst_y,fgPct_y,fg3Pct_y,ftPct_y,tsPct_y,efgPct_y,tovPct_y,orbPct_y,drbPct_y,astTov_y,endDate_y,Predicted Differential
1,36046,38,104196,2346638,2024-02-14 20:00:00+00:00,Samford,88,62,26,6343,5955,1153,1106,2845,668,254,1063,817,2028,1484,2201,4753,745,2042,1196,1629,3063,3228,388,112.020788,105.446404,199.000000,0.463076,0.364838,0.734193,0.579824,0.541448,16.271836,0.287170,0.712830,1.040452,2024-02-14 20:00:00+00:00,36046,38,104450,2024-02-14 20:00:00+00:00,Western Caro.,62,88,-26,6060,6298,854,1018,3109,408,219,951,874,2235,1471,2135,5087,732,2214,1058,1520,2812,3145,-238,104.980200,109.129586,193.729412,0.419697,0.330623,0.696053,0.526426,0.491645,14.179639,0.281119,0.718881,1.070452,2024-02-14 20:00:00+00:00,4.166667
2,36046,38,104450,2346638,2024-02-14 20:00:00+00:00,Western Caro.,62,88,-26,6060,6298,854,1018,3109,408,219,951,874,2235,1471,2135,5087,732,2214,1058,1520,2812,3145,-238,104.980200,109.129586,193.729412,0.419697,0.330623,0.696053,0.526426,0.491645,14.179639,0.281119,0.718881,1.070452,2024-02-14 20:00:00+00:00,36046,38,104196,2024-02-14 20:00:00+00:00,Samford,88,62,26,6343,5955,1153,1106,2845,668,254,1063,817,2028,1484,2201,4753,745,2042,1196,1629,3063,3228,388,112.020788,105.446404,199.000000,0.463076,0.364838,0.734193,0.579824,0.541448,16.271836,0.287170,0.712830,1.040452,2024-02-14 20:00:00+00:00,-4.166667
5,30629,38,104450,1979402,2021-11-21 15:30:00+00:00,Western Caro.,64,53,11,452,476,64,95,245,28,12,98,71,174,119,156,363,76,203,64,103,223,218,-24,103.473410,108.847505,201.333333,0.429752,0.374384,0.621359,0.553487,0.534435,19.355348,0.289796,0.710204,0.969388,2021-11-21 15:30:00+00:00,30629,37,103826,2021-11-21 15:30:00+00:00,Longwood,53,64,-11,290,267,49,55,164,25,8,41,58,106,60,117,268,32,98,24,40,141,149,23,109.994413,98.730955,217.250000,0.436567,0.326531,0.600000,0.507703,0.496269,12.553582,0.353659,0.646341,1.341463,2021-11-21 15:30:00+00:00,-4.866667
6,30629,37,103826,1979402,2021-11-21 15:30:00+00:00,Longwood,53,64,-11,290,267,49,55,164,25,8,41,58,106,60,117,268,32,98,24,40,141,149,23,109.994413,98.730955,217.250000,0.436567,0.326531,0.600000,0.507703,0.496269,12.553582,0.353659,0.646341,1.341463,2021-11-21 15:30:00+00:00,30629,38,104450,2021-11-21 15:30:00+00:00,Western Caro.,64,53,11,452,476,64,95,245,28,12,98,71,174,119,156,363,76,203,64,103,223,218,-24,103.473410,108.847505,201.333333,0.429752,0.374384,0.621359,0.553487,0.534435,19.355348,0.289796,0.710204,0.969388,2021-11-21 15:30:00+00:00,4.866667
9,33533,38,104450,2183580,2022-11-19 11:00:00+00:00,Western Caro.,98,91,7,2451,2724,370,453,1274,175,79,442,390,884,600,848,2114,346,1076,409,602,1120,1283,-273,101.829974,113.454004,181.885714,0.401135,0.321561,0.679402,0.515158,0.482971,15.668869,0.306122,0.693878,1.024887,2022-11-19 11:00:00+00:00,33533,72,103791,2022-11-19 11:00:00+00:00,Lamar University,91,98,-7,1874,2248,369,347,1020,186,95,434,330,690,547,705,1729,152,520,312,450,854,1014,-374,93.386340,112.004013,202.266667,0.407750,0.292308,0.693333,0.486248,0.451706,18.382042,0.323529,0.676471,0.799539,2022-11-19 11:00:00+00:00,4.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [15]:
Stanford_spreads = combined_df[combined_df["teamMarket_x"] == "Stanford"]
Stanford_spreads.sort_values(by="Predicted Differential", ascending=False)

,competitionId_x,conferenceId_x,teamId_x,gameId,gameTime_x,teamMarket_x,ptsScored_x_x,ptsAgst_x_x,ptsDiff_x_x,ptsScored_y_x,ptsAgst_y_x,tovDiff_x,ast_x,reb_x,stl_x,blk_x,tov_x,orb_x,drb_x,pf_x,fgm_x,fga_x,fgm3_x,fga3_x,ftm_x,fta_x,ptsP1_x,ptsP2_x,ptsDiff_y_x,ortg_x,drtg_x,netRankAgst_x,fgPct_x,fg3Pct_x,ftPct_x,tsPct_x,efgPct_x,tovPct_x,orbPct_x,drbPct_x,astTov_x,endDate_x,competitionId_y,conferenceId_y,teamId_y,gameTime_y,teamMarket_y,ptsScored_x_y,ptsAgst_x_y,ptsDiff_x_y,ptsScored_y_y,ptsAgst_y_y,tovDiff_y,ast_y,reb_y,stl_y,blk_y,tov_y,orb_y,drb_y,pf_y,fgm_y,fga_y,fgm3_y,fga3_y,ftm_y,fta_y,ptsP1_y,ptsP2_y,ptsDiff_y_y,ortg_y,drtg_y,netRankAgst_y,fgPct_y,fg3Pct_y,ftPct_y,tsPct_y,efgPct_y,tovPct_y,orbPct_y,drbPct_y,astTov_y,endDate_y,Predicted Differential
20482,33533,74,104290,2187285,2022-12-16 17:00:00+00:00,Stanford,85,40,45,2805,2829,496,556,1540,249,127,624,476,1064,726,1013,2343,304,932,475,708,1271,1519,-24,100.698345,101.788041,110.071429,0.432352,0.326180,0.670904,0.528344,0.497226,19.032978,0.309091,0.690909,0.891026,2022-12-16 17:00:00+00:00,33533,59,103661,2022-12-16 17:00:00+00:00,Green Bay,40,85,-45,2468,2929,490,421,1201,205,113,558,345,856,641,879,2134,202,746,508,720,1165,1303,-461,93.893580,111.330693,219.463415,0.411903,0.270777,0.705556,0.503509,0.459231,18.545600,0.287261,0.712739,0.754480,2022-12-16 17:00:00+00:00,19.533333
28090,36046,74,104290,2334125,2023-11-10 18:00:00+00:00,Stanford,91,73,18,4624,4606,778,949,2391,381,202,906,729,1662,1139,1677,3772,517,1471,753,1108,2161,2448,18,105.110413,104.817012,117.164179,0.444592,0.351462,0.679603,0.542784,0.513123,17.539376,0.304893,0.695107,1.047461,2023-11-10 18:00:00+00:00,36046,61,104174,2023-11-10 18:00:00+00:00,Sacramento St.,73,91,-18,3862,4166,608,720,1947,291,158,727,549,1398,978,1419,3256,398,1145,626,896,1764,2038,-304,101.745881,109.601031,204.779661,0.435811,0.347598,0.698661,0.529006,0.496929,16.608639,0.281972,0.718028,0.990371,2023-11-10 18:00:00+00:00,17.633333
7126,33533,74,104290,2192234,2023-01-25 19:00:00+00:00,Stanford,72,65,7,3486,3520,613,683,1876,301,156,742,584,1292,889,1256,2895,381,1147,593,872,1603,1868,-34,102.190495,103.362693,118.173077,0.433851,0.332171,0.680046,0.531616,0.499655,18.454590,0.311301,0.688699,0.920485,2023-01-25 19:00:00+00:00,33533,111,103454,2023-01-25 19:00:00+00:00,Chicago St.,65,72,-7,3372,3930,636,533,1792,288,129,778,616,1176,1055,1172,2954,401,1277,627,909,1547,1817,-558,97.201572,112.820803,190.169811,0.396750,0.314017,0.689769,0.502689,0.464624,18.828837,0.343750,0.656250,0.685090,2023-01-25 19:00:00+00:00,17.033333
28326,36046,74,104290,2339326,2023-12-17 14:00:00+00:00,Stanford,82,64,18,5169,5126,857,1074,2645,423,218,1003,782,1863,1253,1878,4193,584,1656,829,1221,2429,2710,43,105.341188,104.629224,122.027027,0.447889,0.352657,0.678952,0.546378,0.517529,17.494471,0.295652,0.704348,1.070788,2023-12-17 14:00:00+00:00,36046,61,103716,2023-12-17 14:00:00+00:00,Idaho,64,82,-18,4792,5217,741,768,2120,302,155,829,491,1629,1254,1675,3768,485,1424,957,1264,2182,2561,-425,103.801712,113.093753,216.867647,0.444533,0.340590,0.757120,0.554096,0.508891,16.087216,0.231604,0.768396,0.926417,2023-12-17 14:00:00+00:00,15.266667
35381,36046,74,104290,2348824,2024-01-11 20:00:00+00:00,Stanford,88,84,4,5628,5588,920,1184,2847,449,235,1092,824,2023,1355,2043,4533,640,1787,902,1317,2637,2946,40,105.480559,104.902129,119.350000,0.450695,0.358142,0.684890,0.550418,0.521288,17.600186,0.289427,0.710573,1.084249,2024-01-11 20:00:00+00:00,36046,74,104056,2024-01-11 20:00:00+00:00,Oregon St.,84,88,-4,5060,5585,930,848,2417,448,245,1014,659,1758,1384,1806,4205,457,1418,991,1344,2344,2609,-525,99.541824,110.181077,114.636364,0.429489,0.322285,0.737351,0.527483,0.483829,17.451586,0.272652,0.727348,0.836292,2024-01-11 20:00:00+00:00,14.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
scores = cross_val_score(
pipeline,
X=X_train,
y=y_train,
scoring="neg_mean_squared_error",
cv=4)
nmse = -scores.mean()
np.sqrt(nmse)

11.675680983619545